In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import random

In [32]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key=os.getenv('GOOGLE_API_KEY')
if not api_key:
    raise ('Not key')

In [2]:
MAX_PAGE=2
URL='https://cafef.vn/tai-chinh-ngan-hang.chn'
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    'Accept-Language': 'vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7'
}
response=requests.get(URL,headers=HEADERS,timeout=10)

In [3]:
if response.status_code == 200:
    soup=BeautifulSoup(response.content,'html.parser')
soup

<!DOCTYPE html>

<html lang="vi">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>Tin tức tài chính ngân hàng trong và ngoài nước 24 giờ qua | CafeF</title>
<meta content="Cập nhật liên tục, chính xác các thông tin, sự kiện, chính sách, diễn biến của thị trường tài chính ngân hàng Việt Nam với nhiều tin độc quyền.." name="description"/>
<meta content="kinh tế,tài chính,chứng khoán,vnindex,hnx-index,upcom,hose,cập nhật,thông tin,chính xác,liên tục" name="keywords"/>
<meta content="kinh tế,tài chính,chứng khoán,vnindex,hnx-index,upcom,hose,cập nhật,thông tin,chính xác,liên tục" name="news_keywords"/>
<meta content="Tin tức tài chính ngân hàng trong và ngoài nước 24 giờ qua | CafeF" property="og:title"/>
<meta content="Cập nhật liên tục, chính xác các thông tin, sự kiện, chính sách, diễn biến của thị trường tài chính ngân hàng Việt Nam với nhiều tin độc quyền.." property="og:description"/>
<meta content="article" property="og:type">
<meta content="https:

In [4]:
hiden_input=soup.find('input',{'id':'hdZoneId'})
hiden_input

<input id="hdZoneId" name="hdZoneId" type="hidden" value="18834">
<input id="hdZoneUrl" name="hdZoneUrl" type="hidden" value="tai-chinh-ngan-hang"/>
<input id="hdParentUrl" name="hdParentUrl" type="hidden" value=""/>
<input id="hdZoneParentUrl" name="hdZoneParentUrl" type="hidden" value=""/>
</input>

In [5]:
zoneId=hiden_input.get('value')
zoneId

'18834'

In [6]:
all_link=[]
base_api='https://cafef.vn/timelinelist/{}/{}.chn'
for page in range(1, MAX_PAGE+1):
    api_url=base_api.format(zoneId,page)

    r=requests.get(api_url,headers=HEADERS,timeout=10)
    if r.status_code==200:
        api_soup=BeautifulSoup(r.content,'html.parser')
        links_found = 0

        for a_tag in api_soup.find_all('a',href=True):
            href=a_tag['href']
            if href.endswith('.chn') and len(href)>25:
                full_link = "https://cafef.vn" + href if not href.startswith('http') else href
                if full_link not in all_link:
                    all_link.append(full_link)
time.sleep(1)
print(f'{len(all_link)}')
print(f'{all_link[:5]}')

28
['https://cafef.vn/loi-nhuan-mb-vuot-34000-ty-chu-tich-luu-trung-thai-chia-se-ve-muc-tieu-kinh-doanh-nam-2026-tang-22-tong-tai-san-va-25-tin-dung-188260118091555093.chn', 'https://cafef.vn/giam-doc-khoi-du-lieu-vietcombank-dang-ky-mua-co-phieu-vcb-188260118084824021.chn', 'https://cafef.vn/nhung-ngan-hang-nao-dang-co-lai-suat-tiet-kiem-cao-nhat-thi-truong-188260117170018604.chn', 'https://cafef.vn/msb-hop-tac-ksher-don-dau-lan-song-thanh-toan-xuyen-bien-gioi-tai-viet-nam-188260117124029773.chn', 'https://cafef.vn/ong-nguyen-thanh-nghi-de-nghi-ngan-hang-nha-nuoc-cu-the-hoa-phuong-cham-phat-trien-de-on-dinh-on-dinh-de-phat-trien-188260117104558451.chn']


In [7]:
final_data=[]
for i,url in enumerate(all_link[:5]):
    r=requests.get(url,headers=HEADERS,timeout=10)
    soup=BeautifulSoup(r.content,'html.parser')

    title=soup.find('h1',class_='title')
    title_text=title.text.strip() if title else''

    content_div=soup.find('div',class_='detail-content')
    if not content_div:
        content_div=soup.find('div',class_='contentdetail')
    content_text=''
    if content_div:
        for script in content_div(["script", "style", "iframe"]):
            script.decompose()
        paragraphs = [p.text.strip() for p in content_div.find_all('p') if p.text.strip()]
        content_text = "\n".join(paragraphs)
    if title_text and len(content_text) > 50:
        final_data.append({
            'url':url,
            'title':title_text,
            'content':content_text
        })
    time.sleep(1)
df = pd.DataFrame(final_data)

In [9]:
df.head(5)

,url,title,content
0,https://cafef.vn/loi-nhuan-mb-vuot-34000-ty-ch...,"Lợi nhuận MB vượt 34.000 tỷ, Chủ tịch Lưu Trun...",Tại Hội nghị Quân chính năm 2025 và Đón nhận H...
1,https://cafef.vn/giam-doc-khoi-du-lieu-vietcom...,Giám đốc Khối Dữ liệu Vietcombank đăng ký mua ...,Theo thông báo của Sở Giao dịch Chứng khoán TP...
2,https://cafef.vn/nhung-ngan-hang-nao-dang-co-l...,Những ngân hàng nào đang có lãi suất tiết kiệm...,"Ngày 17-1, theo ghi nhận của phóng viên Báo Ng..."
3,https://cafef.vn/msb-hop-tac-ksher-don-dau-lan...,MSB hợp tác Ksher đón đầu làn sóng thanh toán ...,Thị trường thanh toán xuyên biên giới đang đứn...
4,https://cafef.vn/ong-nguyen-thanh-nghi-de-nghi...,Ông Nguyễn Thanh Nghị đề nghị Ngân hàng Nhà nư...,"Ông Nguyễn Thanh Nghị - Trưởng Ban Chính sách,..."


In [13]:
import re

def clean_code(text):
    text=re.sub(r'[^\w\s.,%()-]',' ',text)
    text=re.sub(r'(Theo|Nguồn):.*$',' ',text)
    text=re.sub(r'\s+',' ',text).strip()
    return text

df['content']=df['content'].apply(clean_code)
df['content'].iloc[0]

'Tại Hội nghị Quân chính năm 2025 và Đón nhận Huân chương Lao động Hạng Ba của Ngân hàng TMCP Quân đội (MB), ông Lưu Trung Thái, Chủ tịch HĐQT MB cho biết, năm 2025, dù đối mặt với nhiều thách thức, MB đã nỗ lực hoàn thành tốt các chỉ tiêu do Đại hội đồng cổ đông đề ra, ghi nhận tăng trưởng tích cực trên cả quy mô, hiệu quả và chất lượng tài sản. Theo đó, tổng tài sản MB đạt 1,6 triệu tỷ đồng, tăng 35% so với năm 2024. Lợi nhuận trước thuế toàn tập đoàn đạt trên 34,2 nghìn tỷ đồng, tăng 18,7% so với năm ngoái. Dư nợ tín dụng vượt mức 1 triệu tỷ đồng, tăng 35% - thể hiện vai trò của MB trong việc đáp ứng nhu cầu phát triển của nền kinh tế, đồng hành cùng doanh nghiệp, người dân và các dự án trọng điểm, đóng góp chung vào sự vận hành thông suốt của thị trường tài chính ngân hàng. Các chỉ số về hiệu quả và an toàn duy trì ổn định. Tỷ lệ nợ xấu được kiểm soát ở mức dưới 1.3%, hoàn thành kế hoạch đề ra. Đă c biê t, trong Kết quả đo lường định giá thương hiệu MB 2025 theo bảng xê p ha ng Bra

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

split=RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=100,
    separators=['\n\n','\n','.',' ','']
)
    
chunk=[]
for index,row in df.iterrows():
    doc = Document(
        page_content=row['content'],
        metadata={
            'source':row['url'],
            'title':row['title']
        }
    )
    text_slpit=split.split_documents([doc])
    chunk.extend(text_slpit)
chunk

[Document(metadata={'source': 'https://cafef.vn/loi-nhuan-mb-vuot-34000-ty-chu-tich-luu-trung-thai-chia-se-ve-muc-tieu-kinh-doanh-nam-2026-tang-22-tong-tai-san-va-25-tin-dung-188260118091555093.chn', 'title': 'Lợi nhuận MB vượt 34.000 tỷ, Chủ tịch Lưu Trung Thái chia sẻ về mục tiêu kinh doanh năm 2026: Tăng 22% tổng tài sản và 25% tín dụng'}, page_content='Tại Hội nghị Quân chính năm 2025 và Đón nhận Huân chương Lao động Hạng Ba của Ngân hàng TMCP Quân đội (MB), ông Lưu Trung Thái, Chủ tịch HĐQT MB cho biết, năm 2025, dù đối mặt với nhiều thách thức, MB đã nỗ lực hoàn thành tốt các chỉ tiêu do Đại hội đồng cổ đông đề ra, ghi nhận tăng trưởng tích cực trên cả quy mô, hiệu quả và chất lượng tài sản. Theo đó, tổng tài sản MB đạt 1,6 triệu tỷ đồng, tăng 35% so với năm 2024'),
 Document(metadata={'source': 'https://cafef.vn/loi-nhuan-mb-vuot-34000-ty-chu-tich-luu-trung-thai-chia-se-ve-muc-tieu-kinh-doanh-nam-2026-tang-22-tong-tai-san-va-25-tin-dung-188260118091555093.chn', 'title': 'Lợi nhu

In [25]:
from langchain_chroma.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

model='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
embeddings=HuggingFaceEmbeddings(model_name=model)

chroma=Chroma.from_documents(
    chunk,
    embeddings,
    persist_directory='./chroma_db_economy'
)

retriever=chroma.as_retriever(search_kwargs={'k':5})

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [30]:
y='Gia co phieu VCB'

context=retriever.invoke(y)

context='\n\n'.join([doc.page_content for doc in context])
context

'.339 cổ phiếu VCB, tương ứng khoảng 0,00051% vốn điều lệ của Vietcombank. Nếu giao dịch được thực hiện thành công, số lượng cổ phiếu VCB mà bà Phùng Nguyễn Hải Yến nắm giữ sẽ tăng lên 52.339 cổ phiếu, tương đương khoảng 0,00063% vốn điều lệ của ngân hàng. Trên thị trường, cổ phiếu VCB của Vietcombank vừa trải qua nhịp tăng giá hiếm có khi tăng 27% kể đầu năm 2026. Với thị giá 73.000 đồng cp, ước tính số tiền bà Phùng Nguyễn Hải Yến cần chi ra để thực hiện giao dịch là khoảng 730 triệu đồng.\n\nTheo thông báo của Sở Giao dịch Chứng khoán TP HCM (HoSE), bà Phùng Nguyễn Hải Yến Phó Tổng Giám đốc kiêm Giám đốc Khối Dữ liệu của Ngân hàng TMCP Ngoại thương Việt Nam đã đăng ký mua 10.000 cổ phiếu VCB theo phương thức khớp lệnh, với mục đích đầu tư. Thời gian dự kiến thực hiện giao dịch là từ ngày 10 02 2026 đến ngày 09 03 2026, thông qua tài khoản giao dịch tại Công ty TNHH Chứng khoán Ngân hàng TMCP Ngoại thương Việt Nam Hiện, bà Phùng Nguyễn Hải Yến hiện đang nắm giữ 42\n\n. Đẩy nhanh và t

In [47]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

model='gemini-2.5-flash'
llm=ChatGoogleGenerativeAI(model=model,temperature=0)
def answer(query):
    context=retriever.invoke(query)
    context='\n\n'.join([doc.page_content for doc in context])

    template='''
    `context{context}

    question{question}
    '''
    prompt=ChatPromptTemplate.from_template(template)

    chain= prompt | llm | StrOutputParser()

    return chain.invoke({'context':context, 'question':query})

In [48]:
y='Gia co phieu VCB'

a=answer(y)

a


'Theo ngữ cảnh, thị giá cổ phiếu VCB là **73.000 đồng/cổ phiếu**.'